In [1]:
import pandas as pd
import datetime
import numpy as np
import os, subprocess
import geopandas as gpd
from shapely.geometry import Point
pd.options.display.float_format='{:,.1f}'.format 

In [2]:
# Get GTFS Directory Location
GTFS_Location0=r"N:\Planning - New File Structure\GIS\Data\GTFS\2020_State_Eagle_Ext"

GTFS_Location=GTFS_Location0+"\\"

In [3]:
# Read in data
city_limits=gpd.read_file(r"N:\Planning - New File Structure\GIS\Data\Political\Boundaries\City_Limits.shp")
city_limits=city_limits.to_crs(epsg=4326)
stop_times=pd.read_csv(GTFS_Location+'stop_times.txt')
stops=pd.read_csv(GTFS_Location+'stops.txt')
stops['Coordinates']=list(zip(stops.stop_lon,stops.stop_lat))
stops['Coordinates']=stops['Coordinates'].apply(Point)
stops_gdf=gpd.GeoDataFrame(stops,crs={'init':'epsg:4326','no_defs':True},geometry='Coordinates')
stops_gdf=gpd.sjoin(stops_gdf,city_limits)
trips=pd.read_csv(GTFS_Location+'trips.txt')
routes=pd.read_csv(GTFS_Location+'routes.txt')
calendar=pd.read_csv(GTFS_Location+'calendar.txt')
stops_gdf.head()

,stop_id,stop_name,stop_lat,stop_lon,Coordinates,index_right,CITY,Acres,OBJECTID,UpdateDate,Shape_Leng,Shape_Area
0,remix_a718d165-cb9f-4fcd-bef4-b66ebc9f7c04,Ballantyne,43.7,-116.4,POINT (-116.382420693864 43.6960804486346),1,Eagle,"19,235.7",0,July 2013,"519,387.1","837,903,136.3"
1,remix_bb759065-08dd-4da7-b7ae-b9230fd7dcf3,Untitled Stop,43.7,-116.4,POINT (-116.351710814165 43.6883459586609),1,Eagle,"19,235.7",0,July 2013,"519,387.1","837,903,136.3"
2,remix_24ec8090-b01e-4694-9442-5a16b3762f7b,Untitled Stop,43.7,-116.3,POINT (-116.342409847636 43.68846664630671),1,Eagle,"19,235.7",0,July 2013,"519,387.1","837,903,136.3"
3,remix_2044d391-cd57-4a10-99b9-dffcefd9124b,Untitled Stop,43.7,-116.3,POINT (-116.336696147919 43.6873091436471),1,Eagle,"19,235.7",0,July 2013,"519,387.1","837,903,136.3"
10,01020293,State & Horseshoe Bend,43.7,-116.3,POINT (-116.316704 43.683519),1,Eagle,"19,235.7",0,July 2013,"519,387.1","837,903,136.3"


In [4]:
# Merge stuff
stop_times_stops=pd.merge(stop_times,stops)
stop_times_stops_trips=pd.merge(stop_times_stops,trips)
full=pd.merge(stop_times_stops_trips, routes)
full=gpd.GeoDataFrame(full,crs={'init':'epsg:4326','no_defs':True},geometry='Coordinates')
full.columns

Index(['trip_id', 'stop_id', 'stop_sequence', 'arrival_time', 'departure_time',
       'timepoint', 'stop_name', 'stop_lat', 'stop_lon', 'Coordinates',
       'block_id', 'route_id', 'service_id', 'direction_id', 'shape_id',
       'agency_id', 'route_short_name', 'route_long_name', 'route_type',
       'route_color', 'route_text_color'],
      dtype='object')

In [5]:
#Dissolve by route and stop_id
stops_routes=full.dissolve(by=['stop_id','route_short_name'])
stops_routes.head()

ValueError: cannot join with no level specified and no overlapping names

In [6]:
routes_stop_count=stops_routes.dissolve(by='route_short_name',aggfunc='count')
routes_stop_count['Stops']=routes_stop_count['trip_id']
routes_stop_count.drop(routes_stop_count.columns.difference(['route_short_name','Stops']),1,inplace=True)
routes_stop_count.head()

NameError: name 'stops_routes' is not defined

In [ ]:
geostops=gpd.sjoin(full,city_limits)
geostops

In [ ]:
stops_cities=geostops.dissolve(['route_short_name','stop_id'])

In [ ]:
stop_cities_count=stops_cities.dissolve(by=['route_short_name','CITY'],aggfunc='count')
stop_cities_count['Stops']=stop_cities_count['trip_id']
stop_cities_count.drop(stop_cities_count.columns.difference(['route_short_name','Stops']),1,inplace=True)
stop_cities_count.head()

In [ ]:
final=stop_cities_count.join(routes_stop_count,on='route_short_name',lsuffix='_city',rsuffix='_total')
final

In [ ]:
# final.to_excel(r"P:\Projects\Professional Services\Local Allocations\No_44_StopsByJurisdiction.xlsx",merge_cells=False)